In [21]:
input_image_file = "img//test1.png"
output_image_file="img//test1_processed.png"
resized_image_file="img//test1_resized.png"

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import random

def generate_greyscale_color():
    grey_value = random.randint(0, 255)
    return (grey_value, grey_value, grey_value)


# Create a DataFrame with random values and variable-length strings
data = {
    'col1':['/'.join([str(random.randint(1000000000, 9000000000)), str(random.randint(10000000, 90000000))]) for _ in range(50)],
    'col2': np.random.randint(0, 100, 50),
    'col3': [''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(5, 15))) for _ in range(50)],
    'col4': np.random.randint(100000000, 900000000, 50),
    'col5': [random.choice(['A', 'B', 'C', 'D']) for _ in range(50)]
}

df_dummy = pd.DataFrame(data)

# Convert the table to an image
fig, ax = plt.subplots(figsize=(10, 10))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=df_dummy.values, colLabels=df_dummy.columns, cellLoc='center', loc='center')

# Adjust table properties
table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(1, 1.5)

# Save the table as an image
table_image_path = 'table.png'
plt.savefig(table_image_path, dpi=300, bbox_inches='tight')
plt.close(fig)

# Add noise to the image
img = Image.open(table_image_path).convert("RGB")  # Ensure the image is in RGB mode
img_array = np.array(img)  # Convert to NumPy array for OpenCV processing

# Add random dots as noise
num_dots = 18000  # Number of dots to add
dot_radius =  random.randint(1, 3)  # Radius of the dots
#dot_color =  generate_greyscale_color()  # Black dots (R, G, B)
dot_color =  (0,0,0)

# Add noise dots to the image
for _ in range(num_dots):
    x = random.randint(0, img_array.shape[1] - 1)  # Random x-coordinate
    y = random.randint(0, img_array.shape[0] - 1)  # Random y-coordinate
    # Draw a filled circle (dot) at the random position
    cv2.circle(img_array, (x, y), dot_radius, dot_color, -1)  # -1 thickness means filled circle

# Save the noisy image
noisy_image_path = 'img\\test1.png'
noisy_img = Image.fromarray(img_array)
noisy_img.save(noisy_image_path)

df_dummy.to_csv("dummy.csv",index=False)

print(f"Noisy table with bigger dots saved as {noisy_image_path}")


Noisy table with bigger dots saved as img\test1.png


In [25]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt

input_image_file = "img//test1.png"
output_image_file="img//test1_processed.png"
 
def apply_median_filter(image,kernel_size=1):
    # Apply a median filter with a 3x3 kernel
    filtered_image = cv2.medianBlur(image, kernel_size)
    return filtered_image

def convert_to_negative(image):
    # Convert the image to its negative
    negative_image = cv2.bitwise_not(image)
    return negative_image

def apply_erosion(image, kernel_size=3, iterations=1):
    # Create a kernel for erosion
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    # Apply erosion to the image
    eroded_image = cv2.erode(image, kernel, iterations=iterations)
    return eroded_image

def apply_dilation(image, kernel_size=3, iterations=1):
    # Create a kernel for dilation
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    # Apply dilation to the image
    dilated_image = cv2.dilate(image, kernel, iterations=iterations)
    return dilated_image

output_img = cv2.imread(input_image_file)
# Example usage:

#output_img = convert_to_negative(output_img)
#output_img = apply_erosion(img,kernel_size=2,iterations=1)
#output_img = apply_dilation(output_img,kernel_size=2,iterations=1)


# Thresholding to create a binary image 
_, binary_image = cv2.threshold(output_img, 127, 255, cv2.THRESH_BINARY) 
 
# Define a kernel 
kernel = np.ones((1, 1), np.uint8) 
 
# Apply morphological opening 
#output_img = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel) 
output_img = cv2.morphologyEx(output_img, cv2.MORPH_OPEN, kernel) 
output_img = apply_median_filter(output_img, kernel_size=3) 

cv2.imwrite(output_image_file, output_img)


True

In [ ]:
import matplotlib.pyplot as plt

img  = cv2.imread(input_image_file)
original_height, original_width = img.shape[:2]
scale_percent=30
# Calculate new dimensions
new_width = int(original_width * scale_percent / 100)
new_height = int(original_height * scale_percent / 100)
    
    # Resize the image
resized_image = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    
cv2.imwrite(resized_image_file, resized_image)    
print(resized_image.shape[:2])

plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct colors
plt.axis('off')  # Hide axes
plt.show()

In [23]:
import tiktoken
import os
import base64
from PIL import Image
import base64
import openai
from pydantic import BaseModel
import pandas as pd
import json
import cv2 


os.environ["OPENAI_API_KEY"]="" # Replace with your OpenAI API key
# Set your OpenAI API key
openai.api_key = os.environ["OPENAI_API_KEY"]


client = openai.OpenAI()
certainty="50%"

SYSTEM_PROMPT = f"""Act as an OCR assistant. Analyze the provided image and:
1. Recognize all visible text in the image as accurately as possible.
2. Maintain the original structure and formatting of the text.
3. If any words or phrases are unclear, indicate this with [unclear] in your transcription.
4. Return the output on json format.
5. Replace any character of the text with # if you are certain less than {certainty}
6. This is a table all 5 columns have to be in the json structure and have to have values
7. Col1 is a list where each element has a format  string / string
8. Col2 is a list where each element has a format length max 2
9. Col3 is a list where each element is a string
10. Col4 is a list where each element is a string of length 9
11. Col5 is a list where each element  is a string of length 1
12. You should not have empty rows and if you are not cetrain at all add ###
Provide only the transcription without any additional comments."""



def calculate_match_percentage(string1, string2):
    """
    Calculate the percentage of overlap between two strings.
    
    Args:
        string1 (str): The smaller or substring to compare.
        string2 (str): The larger string to compare against.

    Returns:
        float: Percentage of characters in string1 found in string2.
    """
    # Check if string1 is a substring of string2
    if string1 in string2:
        return 100.0  # Full match as substring
    
    # Calculate character-wise matches
    match_count = sum(1 for char1, char2 in zip(string1, string2) if char1 == char2)
    
    # Calculate percentage match
    percentage = (match_count / len(string1)) * 100
    return percentage


def encode_image_to_base64(image_path):
    """Convert an image file to a base64 encoded string."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


class OCRData(BaseModel):
    col1: list[str]
    col2: list[str]
    col3: list[str]
    col4: list[str]
    col5: list[str]


#resize image scale
scale_percent=50
model_name = "gpt-4o-mini-2024-07-18" #gpt-4o-2024-08-06, gpt-4o-mini
total_tokens=0

# Getting the base64 string
img  = cv2.imread(input_image_file)

original_height, original_width = img.shape[:2]

# Calculate new dimensions
new_width = int(original_width * scale_percent / 100)
new_height = int(original_height * scale_percent / 100)
    
# Resize the image
resized_image = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

cv2.imwrite(resized_image_file, resized_image)    

base64_image = encode_image_to_base64(resized_image_file)

response = client.beta.chat.completions.parse(
    model=model_name, 
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """extract the data from the table that has 5 columns in the image.""",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        },
    ],
      response_format={ "type": "json_object" }#OCRData,

)


r = json.loads(response.choices[0].message.content)

encoding = tiktoken.encoding_for_model(model_name).name

for col in ["col1","col2","col3","col4","col5"]:
    col_tokens = num_tokens_from_string(" ".join(r[col]),encoding)
    total_tokens = total_tokens + col_tokens

#depends on the model
#price_per_output_token = 10/1000000 
print("image size:",resized_image.shape)
price_per_output_token = 0.600/1000000 
image_cost = 0.005525 #for x=1942, y=1192

print("Total tokens output:",total_tokens,"Total cost:",total_tokens*price_per_output_token+image_cost)


image size: (1942, 1192, 3)
Total tokens output: 1045 Total cost: 0.006152


In [49]:
r = json.loads(response.choices[0].message.content)

ai_original_col1 = r["col1"]
ai_original_col2 = r["col2"]
ai_original_col3 = r["col3"]
ai_original_col4 = r["col4"]
ai_original_col5 = r["col5"]


In [50]:

false_count= true_count=0
k=3
col_name="col5"
ai_col=ai_original_col5

df_dummy = pd.read_csv("dummy.csv")


for i, r in df_dummy.head(len(ai_col)).iterrows():
    if k>=0:
        string1 =  ai_col[i][:k]
        string2 = str(df_dummy[col_name].loc[i])[:k]
        condition =string1.startswith(string2)
        percentage_match = calculate_match_percentage(string1, string2)

    if k<0:
        string1 =  ai_col[i][k:]
        string2 = str(df_dummy[col_name].loc[i])[k:]
        condition =string1.startswith(string2)
        percentage_match = calculate_match_percentage(string1, string2)

    if condition:
        true_count += 1
    else:
        false_count+= 1

print(f"TRUE COUNT:{true_count},{true_count/len(df_dummy)}")
print(f"FALSE COUNT:{false_count},{false_count/len(df_dummy)}")

TRUE COUNT:15,0.3
FALSE COUNT:35,0.7


In [44]:
r

col1    3625663720/10972910
col2                     30
col3           jraysihfhpwb
col4              162135727
col5                      B
Name: 49, dtype: object

In [61]:
TRUE COUNT:33,0.66
FALSE COUNT:16,0.32

TRUE COUNT:34,0.68
FALSE COUNT:16,0.32


50%
TRUE COUNT:32,0.64
FALSE COUNT:18,0.36

30%
TRUE COUNT:18,0.36
FALSE COUNT:32,0.64



Original
TRUE COUNT:35,0.7
FALSE COUNT:15,0.3

26025
76025
